In [1]:
%matplotlib widget
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=Z07DOYC023WXEQJ0WRMZ
%env CLEARML_API_SECRET_KEY=EHPtYtLpKmRTyduuLEcf8mlUiGjXsJtzfkO1HEEOuUA3mC72qp
from PIL import Image
import pylidc as pl
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import random
import shutil

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=Z07DOYC023WXEQJ0WRMZ
env: CLEARML_API_SECRET_KEY=EHPtYtLpKmRTyduuLEcf8mlUiGjXsJtzfkO1HEEOuUA3mC72qp


In [2]:
train_image_path = '../datasets/lidc-idri/images/train/'
train_label_path = '../datasets/lidc-idri/labels/train/'

test_image_path = '../datasets/lidc-idri/images/test/'
test_label_path = '../datasets/lidc-idri/labels/test/'

val_image_path = '../datasets/lidc-idri/images/val/'
val_label_path = '../datasets/lidc-idri/labels/val/'

AIR_DENSITY = -1000
BONE_DENSITY = 400

In [6]:
# Buat generate data training
import os
import pydicom as dicom
import cv2

clahe = cv2.createCLAHE()
dataset = pl.query(pl.Scan).all()

def transform_to_hu(medical_image, image):
    intercept = medical_image.RescaleIntercept
    slope = medical_image.RescaleSlope
    hu_image = image * slope + intercept

    return hu_image
def window_image(image, window_center, window_width):
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    window_image = image.copy()
    window_image[window_image < img_min] = img_min
    window_image[window_image > img_max] = img_max
    
    return window_image

for data in dataset:
        filename = data.patient_id
        images = data.load_all_dicom_images()
        
        dicom_dirpath = data.get_path_to_dicom_files()
        
        nodes = data.cluster_annotations()
        slice_indexes = []
        for node in nodes:
                annot_centroids = []
                annot_bbox_matrices = []
                for annot in node:
                        annot_centroids.append(annot.centroid)
                        annot_bbox_matrices.append(annot.bbox_matrix())
                annot_centroids_tp = list(zip(*annot_centroids))
                annot_bbox_matrices_tp = list(zip(*annot_bbox_matrices))
                
                # Annot_centroid_avg contains x_center, y_center, and image_slice for a nodule discovered within one CT scan
                annot_centroid_avg = [sum(centroid_tp) / len(centroid_tp) for centroid_tp in annot_centroids_tp]
                annot_bbox_matrix_avg = [sum(bbox_tp) / len(bbox_tp) for bbox_tp in annot_bbox_matrices_tp]
                
                # Determine annotation labels
                annot_slice = int(round(annot_centroid_avg[2]))
                annot_x_center = annot_centroid_avg[1]
                annot_y_center = annot_centroid_avg[0]
                annot_width = abs(annot_bbox_matrix_avg[1][1]-annot_bbox_matrix_avg[1][0])
                annot_height = abs(annot_bbox_matrix_avg[0][1]-annot_bbox_matrix_avg[0][0])
                
                # Idenfity Dicom Image metadata
                dicom_path = os.path.join(dicom_dirpath, os.listdir(dicom_dirpath)[annot_slice])
                dicom_image = dicom.read_file(dicom_path)
                rescale_slope = dicom_image.RescaleSlope
                rescale_intercept = dicom_image.RescaleIntercept
                window_centers = dicom_image.WindowCenter
                window_widths = dicom_image.WindowWidth
                
                # Generate image
                image = np.array(images[annot_slice].pixel_array)
                
                # Apply rescaling to get pixel values
                hu_image = transform_to_hu(dicom_image, image)
                hu_image[hu_image < AIR_DENSITY] = AIR_DENSITY
                hu_image[hu_image > BONE_DENSITY] = BONE_DENSITY
                
                if isinstance(window_centers, dicom.multival.MultiValue):
                        window_center = window_centers[0]
                else:
                        window_center = window_centers
                if isinstance(window_widths, dicom.multival.MultiValue):
                        window_width = window_widths[0]
                else:
                        window_width = window_widths
                        windowed_image = window_image(hu_image, window_center, window_width)
                # Rescale to 8-bit range
                min_value = windowed_image.min()
                max_value = windowed_image.max()
                displayed_img = ((windowed_image - min_value) * (255 / (max_value - min_value))).astype(np.uint8)
                
                # # Apply clahe
                # processed_img = clahe.apply(displayed_img)
                
                annot_img = Image.fromarray(displayed_img)
                annot_x_center /= annot_img.width
                annot_y_center /= annot_img.height
                annot_width /= annot_img.width
                annot_height /= annot_img.height
                
                
                with open(train_label_path + filename + '-' + str(annot_slice) + '.txt', 'a') as file:
                        file.write('0 '+str(annot_x_center)+' '+str(annot_y_center)+' '+str(annot_width)+' '+str(annot_height)+'\n')
                
                annot_img.save(train_image_path + filename + '-' + str(annot_slice) + '.png')

Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a 

In [9]:
# Split train images and labels into val and test
os.makedirs(val_label_path, exist_ok=True)
os.makedirs(test_label_path, exist_ok=True)

image_files = os.listdir(train_image_path)

random.shuffle(image_files)

num_total_files = len(image_files)
num_validation_files = int(num_total_files * 0.2)  # Adjust the ratio as needed
num_test_files = int(num_total_files * 0.2)  # Adjust the ratio as needed

train_image_files = image_files[num_validation_files + num_test_files:]
validation_image_files = image_files[:num_validation_files]
test_image_files = image_files[num_validation_files:num_validation_files + num_test_files]

for file in validation_image_files:
    source_path = os.path.join(train_image_path, file)
    dest_path = os.path.join(val_image_path, file)
    shutil.move(source_path, dest_path)

for file in test_image_files:
    source_path = os.path.join(train_image_path, file)
    dest_path = os.path.join(test_image_path, file)
    shutil.move(source_path, dest_path)

for file in validation_image_files:
    annotation_file = os.path.splitext(file)[0] + '.txt'
    source_path = os.path.join(train_label_path, annotation_file)
    dest_path = os.path.join(val_label_path, annotation_file)
    if os.path.exists(source_path):
        shutil.move(source_path, dest_path)

for file in test_image_files:
    annotation_file = os.path.splitext(file)[0] + '.txt'
    source_path = os.path.join(train_label_path, annotation_file)
    dest_path = os.path.join(test_label_path, annotation_file)
    if os.path.exists(source_path):
        shutil.move(source_path, dest_path)

print(f"Moved {len(validation_image_files)} image and annotation files to the 'val' folder.")
print(f"Moved {len(test_image_files)} image and annotation files to the 'test' folder.")


Moved 903 image and annotation files to the 'val' folder.
Moved 903 image and annotation files to the 'test' folder.


In [8]:
# Move all image and label to train folder (Undo data-splitting)

for file in os.listdir(val_image_path):
    source_path = os.path.join(val_image_path, file)
    dest_path = os.path.join(train_image_path, file)
    shutil.move(source_path,dest_path)
    
for file in os.listdir(test_image_path):
    source_path = os.path.join(test_image_path, file)
    dest_path = os.path.join(train_image_path, file)
    shutil.move(source_path,dest_path)
    
test_label_path = 'datasets/lidc-idri/labels/test/'
val_label_path = 'datasets/lidc-idri/labels/val/'
train_label_path = 'datasets/lidc-idri/labels/train/'

for file in os.listdir(val_image_path):
    source_path = os.path.join(val_label_path, file)
    dest_path = os.path.join(train_label_path, file)
    shutil.move(source_path,dest_path)
    
for file in os.listdir(test_label_path):
    source_path = os.path.join(test_label_path, file)
    dest_path = os.path.join(train_label_path, file)
    shutil.move(source_path,dest_path)